In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cf
import numpy as np

In [ ]:
# import fields and grid

%store -r fields
[uWind,vWind,wWind,temp,geop,div,vort,geop_height] = fields

%store -r grid
[lon,lat,pressure_levels] = grid

In [ ]:
# for calculating the gradient of a given field, first the distance between
# grid points is converted to metres. the distance in meridional direction is
# constant (deg_to_m) but varies in zonal direction depending on the latitude (dist)

deg_to_m = 111120

dist = np.array(deg_to_m * np.cos(np.deg2rad(lat)))
dist = np.swapaxes(np.tile(dist,[len(lon),1]),0,1)

def grad(field):
    '''this function calculates the gradient of field'''
    grad = np.zeros((2,*temp.isel(time=0).shape))
    
    for j in range(1,len(field.latitude)-1):
        grad[1,:,j,:] = - 1 / (2 * deg_to_m) * (field.isel(time=1)[:,j+1,:] - field.isel(time=1)[:,j-1,:])
    grad[1,:,0,:] = - 1 / deg_to_m * (field.isel(time=1)[:,1,:] - field.isel(time=1)[:,0,:])
    grad[1,:,-1,:] = - 1 / deg_to_m * (field.isel(time=1)[:,-1,:] - field.isel(time=1)[:,-2,:])
    
    for i in range(1,len(field.longitude)-1):
        grad[0,:,:,i] = 1 / (2 * dist[:,i]) * (field.isel(time=1)[:,:,i+1] - field.isel(time=1)[:,:,i-1])
    grad[0,:,:,0] = 1 / dist[:,0] * (field.isel(time=1)[:,:,1] - field.isel(time=1)[:,:,0])
    grad[0,:,:,-1] = 1 / dist[:,-1] * (field.isel(time=1)[:,:,-1] - field.isel(time=1)[:,:,-2])
    
    return grad

In [ ]:
# calculate and store coriolis parameter

omega = 7.2921e-5
f = np.zeros((len(lat),len(lon)))
f = 2 * omega * np.sin(np.deg2rad(lat))
f = np.swapaxes(np.tile(f, [len(lon),1]),0,1)
%store f

The horizontal momentum equation (neglecting friction) is
\begin{align}
\dfrac{d\pmb{u}_h}{dt} &= - \nabla_p \phi - f (\pmb{k} \times \pmb{u})_h\\
\dfrac{\partial \pmb{u}_h}{\partial t} + \pmb{u}_h \nabla_p \pmb{u}_h &= - \nabla_p \phi + f (v\pmb{i} - u \pmb{j})
\end{align}

In [ ]:
# calculate the individual terms of the momentum equation

# local change of wind velocity
loc_change = [( uWind[2,:,:,:] - uWind[0,:,:,:] ) / 2,
              ( vWind[2,:,:,:] - vWind[0,:,:,:] ) / 2]

# horizontal (i.e. on pressure levels) wind velocity advection
adv_change = [uWind[1,:,:,:],vWind[1,:,:,:]] * (grad(uWind) + grad(vWind)) * 3600

# geopotenial gradient term
geop_term = - grad(geop) * 3600

# coriolis term
cor_term = f * [vWind[1,:,:,:], - uWind[1,:,:,:]] * 3600

In [ ]:
def plot_mom_terms(field,vector,N=90,S=90,W=0,E=360,pressure_level=0,
                   spacing=5,vmin=None,vmax=None,scale=200):
    '''to be used for the scale analysis of the momentum equation.
       this function plots field with an areal extend of [N,S,W,E] at
       pressure_level with the momentum equation term (vector)
       displayed as arrows.
       W is given in degrees east and has to be smaller than E,
       also no negative values are allowed.
       spacing gives the space between arrows in degrees.
       scale gives the length of the arrows in the image. try out
       different values, smaller values give longer arrows.
       vmin and vmax are the minumum and maximum values of field in the colorbar.
       the geopotential height is displayed as contour lines (in m).
    '''

    N = 90-N
    S = 90+S
    
    fig, ax = plt.subplots(figsize=(15,8), subplot_kw={'projection': ccrs.PlateCarree()})
    im = ax.contourf(lon[W:E], lat[N:S], field[1,pressure_level,N:S,W:E],
                    cmap='viridis', levels=50, vmin=vmin, vmax=vmax)
    
    im2 = ax.contour(lon[W:E], lat[N:S], geop_height[1,pressure_level,N:S,W:E])
    ax.clabel(im2, im2.levels, inline=True,colors='k')    

    Q = ax.quiver(lon[W:E][::spacing], lat[N:S][::spacing],
                vector[0][pressure_level,N:S,W:E][::spacing,::spacing],
                vector[1][pressure_level,N:S,W:E][::spacing,::spacing],
                scale = scale)
    Qk = ax.quiverkey(Q,0.5,-0.1,np.nanmax(vector[:,pressure_level,N:S,W:E][::spacing,::spacing]),
                      label="{:.0f}".format(np.nanmax(vector[:,pressure_level,N:S,W:E][::spacing,::spacing]))
                      + "m/s /h",labelpos = 'E')
    ax.add_feature(cf.COASTLINE)
    ax.add_feature(cf.BORDERS)
    ax.set_xticks([0],[0])
    ax.set_yticks([0],[0])

    fig.colorbar(im, orientation='horizontal', fraction=0.039*len(lon)/len(lat),label=f"{field.long_name} [{field.units}]")
    ax.set_title(f"{field.long_name} at pressure level {pressure_levels[pressure_level]} hPa", fontsize=15)
    fig.tight_layout()

### Use the function to display and compare the terms of the momentum equation
also try out different regions and altitudes, and different values for spacing and scale <br>
<br>
the variables are <br>
local change of wind velocity: loc_change <br>
horizontal (i.e. on pressure levels) wind velocity advection: adv_change <br>
geopotenial gradient term: geop_term <br>
coriolis term: cor_term <br>

In [ ]:
pressure_levels # in hPa

In [ ]:
plot_mom_terms(temp,adv_change,N=70,S=0,W=270,E=360,pressure_level=3,spacing=2,scale=50)

In [ ]:
plot_mom_terms(uWind,adv_change,pressure_level=2,spacing=5,scale=200)